In [ ]:
# 获得下一个点的二维坐标，并移动
def move_to_next(next_loction_X, next_loction_Y):
    Y,X,direction = XY_Robot()
    # 转向
    move_dir = [next_loction_X-X, next_loction_Y-Y]
    norm_move_dir = move_dir / np.linalg.norm(move_dir)
    rotate_to_balance(norm_move_dir)
    # 向前走！
    run_action('Forwalk02')
    wait_req()